In [203]:
# Final Group Project
# QNT755 Summer 2022
# Dataset:  WHO_COVID_19_global_data.csv
# Datafields:
# Country
# YrMo
# Rep_Dt
# New_cases
# New_deaths
# Tot_Cases
# Tot_Dths
# Vax_Rate
# 1stVaxDt
# Vax_Type_Cnt

# Questions to answer:

# What 10 countries had the highest death rate?
# What 10 countries had the highest vaccination rate? 
# What 5 countries had the highest death rates pre and post vaccination?
# What country had the highest number of vaccination types available?
# What countries had the fewest number of vaccinations available?

In [204]:
import pandas as pd
import numpy as np
import csv

# create new variable showing vaccination available = 4+ and vaccination available under 4
def typ_grp(Vax_Type_Cnt):
    if (Vax_Type_Cnt > 3):
        print('4+')
    else:
        print('Under 4')

# scrubbing data
covid_df = pd.read_csv('WHO_COVID_19_global_data.csv')
covi_df = pd.DataFrame(covid_df)
covi_df['Vax_Typ_Grp'] = np.where(covi_df['Vax_Type_Cnt'] > 3, '4+', 'Under 4') #create new column grouping vax varieties
covi_df[['Mo', 'Day', 'Yr']] = covi_df['Date_reported'].str.split('/', expand=True) #split date field
covi_df['YrMo'] = covi_df['Yr'].map(str) + '/' + covi_df['Mo'].map(str) #concat just year and month fields
covi_df2 = covi_df.drop(['Country_code', 'WHO_region', 'Vax_Type_Cnt', 'Day', 'Mo'], axis=1) #drop uneeded columns
covi_df2.rename(columns = {'Date_reported':'Rep_Dt', 'Cumulative_deaths':'Tot_Dths', 'Cumulative_cases':'Tot_Cases', 'First_Vax_Date':'1stVaxDt'}, inplace = True) #rename columns
covi_df2 =covi_df2[["Country", "YrMo", "Rep_Dt", "New_cases", "New_deaths", "Tot_Cases", "Tot_Dths", "Vax_Rate", "1stVaxDt", "Vax_Typ_Grp"]] #reorder columns
covi_df2['Vax_PrePost'] = np.where(covi_df2['Rep_Dt'] > covi_df2['1stVaxDt'], 'Post_Vax', 'Pre_Vax') #create new column to sort data by pre/post first vax release

print(covi_df2)

            Country    YrMo     Rep_Dt  New_cases  New_deaths  Tot_Cases  \
0       Afghanistan  2020/1   1/3/2020          0           0          0   
1       Afghanistan  2020/1   1/4/2020          0           0          0   
2       Afghanistan  2020/1   1/5/2020          0           0          0   
3       Afghanistan  2020/1   1/6/2020          0           0          0   
4       Afghanistan  2020/1   1/7/2020          0           0          0   
...             ...     ...        ...        ...         ...        ...   
214243     Zimbabwe  2022/6  6/20/2022         30           0     254831   
214244     Zimbabwe  2022/6  6/21/2022         88           2     254919   
214245     Zimbabwe  2022/6  6/22/2022          0           0     254919   
214246     Zimbabwe  2022/6  6/23/2022          0           0     254919   
214247     Zimbabwe  2022/6  6/24/2022          0           0     254919   

        Tot_Dths  Vax_Rate   1stVaxDt Vax_Typ_Grp Vax_PrePost  
0              0     12

In [217]:
# What 10 countries had the highest death rate?
cases_deaths_by_country = covi_df2.groupby(['Country'])[['New_cases', 'New_deaths']].apply(sum) # added up new cases and new deaths and grouped by country
print(cases_deaths_by_country) #results - used to find death rate as well

cases_deaths_by_country['death_rate_by_country(%)']=cases_deaths_by_country['New_deaths']/ cases_deaths_by_country['New_cases']*100

pd.options.display.float_format = '{:.2f}'.format

deathrate = cases_deaths_by_country.sort_values(by=['death_rate_by_country(%)'],ascending=False)

print('Countries with the highest death rate:')
deathrate.head(10)

                                                    New_cases  New_deaths
Country                                                                  
Afghanistan                                            181934        7715
Albania                                                278211        3497
Algeria                                                266015        6875
American Samoa                                           6414          31
Andorra                                                 43774         153
...                                                       ...         ...
Wallis and Futuna                                         454           7
Yemen                                                   11824        2149
Zambia                                                 324334        3997
Zimbabwe                                               254919        5536
occupied Palestinian territory, including east ...     658503        5660

[237 rows x 2 columns]
Countries with

,New_cases,New_deaths,death_rate_by_country(%)
Country,,,
Yemen,11824,2149,18.17
Sudan,62551,4951,7.92
Peru,3605614,213425,5.92
Syrian Arab Republic,55918,3150,5.63
Mexico,5906953,325487,5.51
Somalia,26748,1361,5.09
Egypt,514047,24722,4.81
Afghanistan,181934,7715,4.24
Bosnia and Herzegovina,378370,15799,4.18


In [218]:
# What 10 countries had the highest vaccination rate?
VaxRate = covi_df2[['Country','Vax_Rate']]
Vax_Rate = VaxRate.drop_duplicates()
vaccinationrate = Vax_Rate.sort_values(by=['Vax_Rate'],ascending=False)
print('Countries with the highest vaccination rate:')
vaccinationrate.head(10)

Countries with the highest vaccination rate:


,Country,Vax_Rate
72320,Gibraltar,122.94
144640,Palau,99.33
193456,Tokelau,99.04
202496,United Arab Emirates,99.01
27120,Brunei Darussalam,97.55
152776,Puerto Rico,93.28
194360,Tonga,91.92
37064,Chile,91.83
153680,Qatar,91.75
34352,Cayman Islands,90.45


In [220]:
# What 5 countries had the highest death rates pre and post vaccination?

deaths_by_prepost = covi_df2.groupby(['Country', 'Vax_PrePost'])[['New_cases', 'New_deaths']].apply(sum) # added up new cases and new deaths and grouped by country
#print(deaths_by_prepost) #results - used to find death rate as well
deaths_by_prepost['death_rate_by_country(%)']=deaths_by_prepost['New_deaths']/ deaths_by_prepost['New_cases']*100

pd.options.display.float_format = '{:.2f}'.format

prepost = deaths_by_prepost.sort_values(by=['death_rate_by_country(%)'],ascending=False)
print('Countries with highest death rates pre and post vaccination:')
prepost.head(5)


Countries with highest death rates pre and post vaccination:


New_cases  New_deaths  death_rate_by_country(%)
Country Vax_PrePost                                                 
Yemen   Post_Vax          5825        1272                     21.84
        Pre_Vax           5999         877                     14.62
Peru    Post_Vax       1769186      163253                      9.23
Sudan   Post_Vax         20584        1641                      7.97
        Pre_Vax          41967        3310                      7.89

In [221]:
Covid_Tbl = pd.pivot_table(covi_df3, values=['New_cases','New_deaths'], index=['YrMo'],
                     aggfunc=np.sum, fill_value=0)
# Covid_Tbl

In [222]:
Covid_Tbl2 = pd.pivot_table(covi_df2, values=['New_cases','New_deaths'], index=['YrMo'], columns=['Vax_Typ_Grp'],
                     aggfunc=np.sum, fill_value=0)
#Covid_Tbl2

In [223]:
# What country had the highest number of vaccination types available?
# What countries had the fewest number of vaccinations available?
Cov_Qs = covi_df.groupby('Country')['Vax_Type_Cnt'].mean()
Cov_Q1_Q2 = pd.DataFrame(Cov_Qs)
Cov_Q1 = Cov_Q1_Q2.sort_values(by='Vax_Type_Cnt', ascending=False)
Cov_Q2 = Cov_Q1_Q2.sort_values(by=['Vax_Type_Cnt'])

In [224]:
print('Country with the highest number of vaccine types:')
Cov_Q1.head(1)

Country with the highest number of vaccine types:


,Vax_Type_Cnt
Country,
Iran (Islamic Republic of),12.00


In [225]:
print('Countries with the fewest number of vaccine types:')
Cov_Q2.head()

Countries with the fewest number of vaccine types:


,Vax_Type_Cnt
Country,
Saba,0.00
Sint Eustatius,0.00
Bonaire,0.00
Montserrat,1.00
Chad,1.00
